# Installation and import of necessary packages

In [8]:
!pip install rich==6.2.0
!pip install cobra
import cobra.test
from cobra import Model, Reaction, Metabolite
from cobra.util.solver import linear_reaction_coefficients
from cobra.flux_analysis import flux_variability_analysis
from cobra import sampling

     |████████████████████████████████| 150 kB 4.2 MB/s 
     |████████████████████████████████| 51 kB 5.6 MB/s 
     |████████████████████████████████| 2.4 MB 4.3 MB/s 
     |████████████████████████████████| 109 kB 53.2 MB/s 
     |████████████████████████████████| 7.3 MB 30.4 MB/s 
     |████████████████████████████████| 44 kB 2.1 MB/s 
     |████████████████████████████████| 2.3 MB 38.7 MB/s 
     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 147 kB 56.7 MB/s 
     |████████████████████████████████| 10.9 MB 47.6 MB/s 
     |████████████████████████████████| 216 kB 54.9 MB/s 
     |████████████████████████████████| 68 kB 4.2 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 79 kB 5.5 MB/s 
     |████████████████████████████████| 546 kB 39.4 MB/s 
  Attempting uninstall: rich
    Found existing installation: rich 6.2.0
    Uninstalling rich-6.2.0:
      Successfully uninstalled rich-6.2.0

# Model creation

In [13]:
model = Model('hd4_3')

A = Metabolite('A')
B = Metabolite('B')
C = Metabolite('C')

# Adding "uptake" reaction b1 : -> A
reaction = Reaction('b1')
reaction.add_metabolites({A: -1.0}) # This enables influx of A into the network
reaction.lower_bound = -1000.
model.add_reaction(reaction)

# Add V1 : A -> B to the network
reaction = Reaction('V1')
reaction.add_metabolites({A: -1.0, B: 1.0})
model.add_reaction(reaction) # default lower bounds=0 and upper bounds=1000 (forward reaction)

# Add V2 : A -> C
reaction = Reaction('V2')
reaction.add_metabolites({A: -1.0, C: 1.0})
model.add_reaction(reaction)

# Add V3 : C -> A
reaction = Reaction('V3')
reaction.add_metabolites({C: -1.0, A: 1.0})
model.add_reaction(reaction)

# Add V4 : C -> B
reaction = Reaction('V4')
reaction.add_metabolites({C: -1.0, B: 1.0})
model.add_reaction(reaction)

# Adding "secretion" reaction b2 : B ->
reaction = Reaction('b2')
reaction.add_metabolites({B: -1.0})
model.add_reaction(reaction)

# Adding "secretion" reaction b3 : C ->
reaction = Reaction('b3')
reaction.add_metabolites({C: -1.0})
model.add_reaction(reaction)

print('%i reactions' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))
print('%i genes' % len(model.genes))

print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


7 reactions
3 metabolites
0 genes
Reactions
---------
b1 : A <=> 
V1 : A --> B
V2 : A --> C
V3 : C --> A
V4 : C --> B
b2 : B --> 
b3 : C --> 


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# Question A) 
## To determine the flux values corresponding to maximizing the current objective (perform FBA), type:

In [12]:
sol = model.optimize()
sol.objective_value
# Flux values are in sol.fluxes, the objective in sol.objective_value and solver status in sol.status.



0.0

Why is the objective value 0.0?

# Question B) 
## Optimize production of compound B by maximizing flux through b2.

In [ ]:
# First, change objective to b2:
model.objective ='b2'

# Create a copy of the model (for question G, not important now!)
model_g = model.copy()

# Now, optimize model again to get the maximized b2 value. Instead of doing 
# model.optimize(), it is also possible to get the current objective value 
# (from FBA) and the uptake/secretion rates of all metabolites by typing:
model.summary()




Why is the objective value 1000?

# Question C)
## Assume that the flux in b1 has been measured experimentally and that b1=14 mmol/gDW/h. To modify the corresponding flux bounds use

In [ ]:
model.reactions.get_by_id("b1").lower_bound = -14 # code to change reaction 
# lower bound

# Find the optimal value as before, but this time look at the individual fluxes
# of the system:
sol = model.optimize()
sol.fluxes

How can you interpret the solution w.r.t. stochiometry (i.e. the S-matrix). Is there a need to produce C? Do there exist other solutions which are equally good as this one?

# Question D) 
## We can probe the „flexibility“ of the system by inspecting the minimum and maximum possible fluxes in individual reactions while simultaneously keeping the objective at maximum (here: the flux in B2). This we do by performing flux variability analysis (FVA).

In [ ]:
#Analyze min/max fluxes (a form of sensitivity analysis)
fva_result = cobra.flux_analysis.flux_variability_analysis(model)

#returns a dictionary af min/max values. If you have the pandas package installed you can use:
import pandas as pd
pd.DataFrame.from_dict(fva_result).round(5)


The reaction b3 „competes“ with b2. What effect does flux in b3 have on the maximization of the objective? When you have answered this question, use *cobrapy* to maximize flux in b2 with the side-constraint that flux in b3 goes from 0 to b3_max. (Hint: this can be done using either a simple for-loop which steps from 0 to b3_max_possible_value , changes lower/upper bounds of b3 and maximizes b2)

In [ ]:
import matplotlib.pyplot as plt # for plotting
max_vals = []
test_bounds = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
for i in test_bounds:
  model2 = model.copy() # Create a copy of the original model to not affect
  # original one...
  model2.reactions.get_by_id("b3").lower_bound = test_bounds[i]
  sol_tmp = model2.optimize()
  max_vals.append(sol_tmp.objective_value)

plt.plot(test_bounds, max_vals)
plt.gca().set(title='Production envelope', ylabel='Max b2',xlabel = 'b3 lower bound');


# Question E)
## Measurements with a mass spec analyzer show that C is secreted by the cell in an amount which corresponds to flux in b3 of 5.5 mmol/gDW/h. How can you add this constraint to the model? (This constraint corresponds to further constraining of the flux space).

In [ ]:
# insert code here

How are the flux values (w.r.t. maximization of b2) affected?

# Question F)
## We now want to study the effects of a gene knockout which disables V1.

In [ ]:
# To simulate a knockout simply fix the flux in v1 to zero:
model.reactions.get_by_id("V1").lower_bound = 0
model.reactions.get_by_id("V1").upper_bound = 0


What is the effect of the knockout on flux values on objective value (assuming maximization of B2)? Additional question: Was there anything preventing the optimizer finding this solution earlier (i.e. before we blocked V1)?

In [ ]:
# insert code here

# Question G)
## To produce a valuable compound D in the cell, a gene is inserted which codes for an enzyme catalyzing the following reaction:
# 2B --> D

## which is called "V5". 

In [ ]:
# Add V3 : 2B -> D
D = Metabolite('D')
reaction = Reaction('V5')
reaction.add_metabolites({B: -2.0, D: 1.0})
model_g.add_reaction(reaction)

# Adding "secretion" reaction b4 : D ->
reaction = Reaction('b4')
reaction.add_metabolites({D: -1.0})
model_g.add_reaction(reaction)


How does this affect the flux values of the model? Use *cobrapy* to study how the objective function changes when flux through b4 is increased.

In [ ]:
# insert code

How can you maximize the production of D? What flux values correspond to this objective?

In [ ]:
# insert code